In [ ]:
# First, set up environment in this notebook session
!pip install --upgrade --quiet openai-agents

import sys
import subprocess

def check_install(pkg):
    res = subprocess.run([sys.executable, "-m", "pip", "show", pkg], stdout=subprocess.PIPE, text=True)
    if "Name: " in res.stdout:
        print(f"{pkg} is installed.\n{res.stdout}")
    else:
        print(f"{pkg} is NOT installed in this environment.")

check_install("openai-agents")

try:
    import openai_agents
    print("Successfully imported openai_agents!")
except ImportError as e:
    print("ImportError =>", e)


openai-agents is installed.
Name: openai-agents
Version: 0.0.14
Summary: OpenAI Agents SDK
Home-page: https://github.com/openai/openai-agents-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: griffe, mcp, openai, pydantic, requests, types-requests, typing-extensions
Required-by: 

ImportError => No module named 'openai_agents'


In [ ]:
import os

# (Optional) set your keys here or skip if you prefesk-proj-MAWj0jq026pbyiyXcqDGT3BlbkFJwAKGtQdUBrbxC49caia4r .env
os.environ["OPENAI_API_KEY"] = "sk-proj-MAWj0jq026pbyiyXcqDGT3BlbkFJwAKGtQdUBrbxC49caia4"
os.environ["OPENAI_MODEL"]   = "o4-mini"  # or another valid model
os.environ["FLASK_SECRET_KEY"] = "R679A"
os.environ["CAPTCHA_API_KEY"] = "c910c1d12f66dc0bf4dd6c293dca8865"
os.environ["CHROMEDRIVER_PATH"] = "/usr/bin/chromedriver"
os.environ["NGROK_AUTH_TOKEN"]    = "2tedizGJDmEJMDtfhCR6ftx8EyX_2RaA7t8nPn17SU7jiyK1H"
print("Environment variables set.")


Environment variables set.


In [ ]:
pip install --upgrade git+https://github.com/openai/openai-agents-python.git

  Cloning https://github.com/openai/openai-agents-python.git to /tmp/pip-req-build-umcvor2e
  Running command git clone --filter=blob:none --quiet https://github.com/openai/openai-agents-python.git /tmp/pip-req-build-umcvor2e
  Resolved https://github.com/openai/openai-agents-python.git to commit f9763495b86afcf0c421451a92200e1141fa8dcb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
pip install flask[async] flask_socketio requests nest_asyncio torch pyngrok

In [ ]:
!pip show openai-agents
!pip freeze | grep openai

Name: openai-agents
Version: 0.0.14
Summary: OpenAI Agents SDK
Home-page: https://github.com/openai/openai-agents-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: griffe, mcp, openai, pydantic, requests, types-requests, typing-extensions
Required-by: 
openai==1.76.2
openai-agents==0.0.14


In [ ]:
import nest_asyncio
nest_asyncio.apply()

from agents import Agent, Runner
import asyncio

buddy_agent = Agent(
    name="buddy_agent",
    instructions="You are a buddy agent. Provide short replies referencing your mood or state."
)
buddy_agent.model = "o4-mini"

async def test_buddy():
    result = await Runner.run(buddy_agent, "Hello from user!")
    print("LLM says:", result.final_output)

# Jupyter can do an `await` at top-level:
await test_buddy()

LLM says: Hey there! I’m feeling upbeat today 😊. How about you?


In [ ]:
#!/usr/bin/env python3
"""
ai_autonomous_agents_blow_latency.py

- 4 Agents with minimal-latency loops: partial MCTS => frequent chart updates,
  Poisson burst+decay emotional states => chaotic but stable,
  deeper cross-talk => each agent can reply back again for a more conversation-like dynamic.
- The dashboard <title> changed to "AI Autonomous Agent".
- If NGROK_AUTH_TOKEN is set => show the public link. Otherwise local only.

Usage:
  python ai_autonomous_agents_blow_latency.py
"""

# ───────────────────────── ASGI Safety Shim ─────────────────────────
import sys, types
if 'asgiref' not in sys.modules:
    stub = types.ModuleType('asgiref')
    sync = types.ModuleType('asgiref.sync')
    def _noop(obj):
        return obj
    sync.sync_to_async = _noop
    sync.async_to_sync = _noop
    stub.sync = sync
    sys.modules['asgiref'] = stub
    sys.modules['asgiref.sync'] = sync
# ─────────────────────────────────────────────────────────────────────

import os
import time
import random
import math
import logging
import asyncio
import threading
import socket
from collections import defaultdict, deque
from enum import Enum
from urllib.parse import urlparse

import requests
import nest_asyncio
nest_asyncio.apply()

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from flask import Flask, request, jsonify, redirect, url_for
from flask_socketio import SocketIO
import socketio

##############################################################################
# Optional Agents SDK
##############################################################################
try:
    from agents import Agent, Runner
    HAS_AGENTS_SDK = True
except ImportError:
    HAS_AGENTS_SDK = False

##############################################################################
# Logging
##############################################################################
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format="[%(asctime)s] [%(levelname)s] %(message)s"
)
logger = logging.getLogger("MainServer")

##############################################################################
# Poisson burst+decay model => 9 states
##############################################################################
class BurstDecay:
    def __init__(self, decay_k=0.8, base_rate=0.4, amp_range=(0.5,1.0)):
        self.decay_k   = decay_k
        self.rate      = base_rate
        self.amp_range = amp_range
        self.intensity = 0.0
        self.last_t    = time.time()

    def maybe_burst(self, context=1.0):
        now= time.time()
        dt= now - self.last_t
        self.last_t= now

        # Exponential decay
        self.intensity*= math.exp(-self.decay_k*dt)

        # Poisson burst
        p= 1 - math.exp(-self.rate*context*dt)
        if random.random()< p:
            amp= random.uniform(*self.amp_range)
            self.intensity+= amp

        return self.intensity

class CogState(Enum):
    CURIOUS       = 1
    ANALYTICAL    = 2
    CAUTIOUS      = 3
    SOCIAL        = 4
    TASK_ORIENTED = 5
    FRUSTRATED    = 6
    CREATIVE      = 7
    NEUROTIC      = 8
    HUMOR         = 9

class BurstManager:
    def __init__(self):
        self.states= list(CogState)
        self.map= {}
        for st in self.states:
            dk= random.uniform(0.6,1.0)
            rt= random.uniform(0.2,0.5)
            self.map[st]= BurstDecay(dk, rt)

    def step(self, context=1.0):
        best_st= None
        best_val= -9999
        for st in self.states:
            val= self.map[st].maybe_burst(context)
            if val> best_val:
                best_val= val
                best_st= st
        return best_st, best_val

##############################################################################
# Global data => for the dashboard
##############################################################################
task_queue= deque()

# agent_logs => store each agent's data
agent_logs= defaultdict(lambda:{
    "current_url":"?",
    "cognitive_state":"?",
    "buddy_msgs":[],
    "cumulative_reward":0.0
})
mcts_history= defaultdict(list)
emo_history= defaultdict(list)

DOMAIN_HOP_REWARD=10.0
MAX_DOMAIN_VISITS=3

PERSONALITIES={
    "agent_1":"Comedian",
    "agent_2":"Philosopher",
    "agent_3":"Scientist",
    "agent_4":"Historian",
}

buddy_agents={}
if HAS_AGENTS_SDK and os.getenv("OPENAI_API_KEY"):
    from agents import Agent, Runner
    for a_id, pers in PERSONALITIES.items():
        ag= Agent(
            name=a_id,
            instructions=f"You are a {pers} in a group of 4. Provide deeper cross-talk referencing multiple agent messages if relevant."
        )
        ag.model= os.getenv("OPENAI_MODEL","o4-mini")
        buddy_agents[a_id]= ag
    logger.info("buddy_agents => created for each personality with Agents SDK.")
else:
    logger.info("No Agents or no OPENAI_API_KEY => fallback buddy text only.")

##############################################################################
# Flask + SocketIO => "AI Autonomous Agent" in the HTML
##############################################################################
app= Flask(__name__)
app.config["SECRET_KEY"]= os.getenv("FLASK_SECRET_KEY","secret!")
server_socketio= SocketIO(app, cors_allowed_origins="*", async_mode="threading")

DASHBOARD_HTML= r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8"/>
<title>AI Autonomous Agent</title>
<script src="https://cdn.socket.io/4.5.4/socket.io.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<style>
.chart-container{display:flex;flex-direction:row;justify-content:space-between}
.chart-box{width:45%}
table{border-collapse:collapse;width:100%;margin-top:1rem;font-size:0.9rem}
th,td{border:1px solid #ccc;padding:6px;text-align:left}
.forms-section{border:1px solid #ccc;padding:1rem;margin-bottom:1rem;background:#f9f9f9}
.forms-section h3{margin-top:0}
</style>
</head>
<body>
<h1>AI Autonomous Agent</h1>
<p>
<a href="/nav_map" target="_blank">Navigation Map (Placeholder)</a> |
<a href="/logs_data" target="_blank">Logs/JSON</a>
</p>
<div class="forms-section">
  <h3>Tasks & Buddy Interaction</h3>
  <form id="taskF" onsubmit="addTask(event)">
    <select id="taskSel">
      <option value="navigate">navigate</option>
      <option value="instruction">instruction</option>
      <option value="upload">upload</option>
    </select>
    <input type="text" id="taskVal" placeholder="URL or content..."/>
    <button>Add Task</button>
  </form>
  <br/>
  <form id="buddyF" onsubmit="sendBuddy(event)">
    <input type="text" id="bf" placeholder="agent_1"/>
    <input type="text" id="bt" placeholder="agent_2"/>
    <input type="text" id="bm" placeholder="Buddy message..."/>
    <button>Send Buddy</button>
  </form>
</div>
<div class="chart-container">
  <div class="chart-box">
    <h2>MCTS / CumulativeReward</h2>
    <canvas id="mctsChart"></canvas>
  </div>
  <div class="chart-box">
    <h2>Emotion Over Time</h2>
    <canvas id="emoChart"></canvas>
  </div>
</div>
<h2>Agent Logs</h2>
<table id="logTable">
  <thead>
    <tr><th>Agent</th><th>URL</th><th>State</th><th>Reward</th><th>Buddy</th></tr>
  </thead>
  <tbody></tbody>
</table>
<script>
let sock= io("/", {path:"/socket.io/"});
let mctsChart, emoChart;

function initCharts(){
  let mc= document.getElementById("mctsChart").getContext("2d");
  mctsChart= new Chart(mc,{
    type:"line",
    data:{datasets:[
      {label:"agent_1",data:[],borderColor:"red",fill:false},
      {label:"agent_2",data:[],borderColor:"blue",fill:false},
      {label:"agent_3",data:[],borderColor:"green",fill:false},
      {label:"agent_4",data:[],borderColor:"orange",fill:false}
    ]},
    options:{responsive:true,scales:{x:{type:"linear"},y:{beginAtZero:true}}}
  });
  let ec= document.getElementById("emoChart").getContext("2d");
  emoChart= new Chart(ec,{
    type:"line",
    data:{datasets:[
      {label:"agent_1",data:[],borderColor:"red",fill:false},
      {label:"agent_2",data:[],borderColor:"blue",fill:false},
      {label:"agent_3",data:[],borderColor:"green",fill:false},
      {label:"agent_4",data:[],borderColor:"orange",fill:false}
    ]},
    options:{responsive:true,scales:{x:{type:"linear"},y:{min:-1,max:1}}}
  });
}
function getM(a){return mctsChart.data.datasets.find(ds=>ds.label===a);}
function getE(a){return emoChart.data.datasets.find(ds=>ds.label===a);}

sock.on("connect",()=>console.log("Dashboard connected"));
sock.on("disconnect",()=>console.log("Dashboard disconnected"));

sock.on("agent_interaction",(d)=>{
  console.log("buddy =>",d);
  setTimeout(refresh,500);
});
sock.on("update_agent_logs",(d)=>{
  updateRow(d.agent_id,d.current_url,d.cognitive_state,d.cumulative_reward);
});
sock.on("emo_update",(d)=>{
  let ds= getE(d.agent_id); if(!ds)return;
  ds.data.push({x:ds.data.length,y:d.emo});
  emoChart.update();
});
sock.on("mcts_decision",(d)=>{
  let ds= getM(d.agent_id); if(!ds)return;
  ds.data.push({x:d.x,y:d.cumulative_reward});
  mctsChart.update();
});

function refresh(){
  fetch("/logs_data")
    .then(r=>r.json())
    .then(js=>{
      let tb= document.getElementById("logTable").querySelector("tbody");
      tb.innerHTML="";
      for(let aid in js.agents){
        let info= js.agents[aid];
        let row= document.createElement("tr");
        row.setAttribute("data-agent",aid);
        for(let i=0;i<5;i++){ row.insertCell(i); }
        row.cells[0].textContent= aid;
        row.cells[1].textContent= info.current_url;
        row.cells[2].textContent= info.cognitive_state;
        row.cells[3].textContent= info.cumulative_reward.toFixed(2);
        let c= row.cells[4];
        info.buddy_msgs.slice(-5).forEach(msgArr=>{
          let st= new Date(msgArr[0]*1000).toLocaleTimeString();
          let dv= document.createElement("div");
          dv.textContent= `[${st}] ${msgArr[1]}`;
          c.appendChild(dv);
        });
        tb.appendChild(row);
      }
      // MCTS
      for(let ds of mctsChart.data.datasets){ds.data=[];}
      for(let a in js.mcts_history){
        let ds= getM(a); if(!ds)continue;
        js.mcts_history[a].forEach(pt=> ds.data.push({x:pt[0],y:pt[1]}));
      }
      mctsChart.update();
      // EMO
      for(let ds of emoChart.data.datasets){ds.data=[];}
      for(let a in js.emo_history){
        let ds= getE(a); if(!ds)continue;
        js.emo_history[a].forEach((pt,i)=> ds.data.push({x:i,y:pt[1]}));
      }
      emoChart.update();
    });
}
function updateRow(aid,url,st,rew){
  let tb= document.getElementById("logTable").querySelector("tbody");
  let tr= tb.querySelector(`tr[data-agent="${aid}"]`);
  if(!tr){
    tr= document.createElement("tr");
    tr.setAttribute("data-agent",aid);
    for(let i=0;i<5;i++){ tr.insertCell(i);}
    tb.appendChild(tr);
  }
  tr.cells[0].textContent= aid;
  tr.cells[1].textContent= url;
  tr.cells[2].textContent= st;
  tr.cells[3].textContent= rew.toFixed(2);
}

function addTask(e){
  e.preventDefault();
  let sel= document.getElementById("taskSel");
  let val= document.getElementById("taskVal").value.trim();
  if(!val)return;
  let data={type:sel.value, content:val};
  fetch("/add-task",{
    method:"POST",
    headers:{"Content-Type":"application/json"},
    body: JSON.stringify(data)
  }).then(r=>r.json()).then(js=>{
    alert("AddTask => "+ JSON.stringify(js));
  });
}

function sendBuddy(e){
  e.preventDefault();
  let f= document.getElementById("bf").value.trim();
  let t= document.getElementById("bt").value.trim();
  let m= document.getElementById("bm").value.trim();
  if(!f||!t||!m)return;
  fetch("/agent_interaction",{
    method:"POST",
    headers:{"Content-Type":"application/json"},
    body: JSON.stringify({from_agent:f,to_agent:t,message:m})
  }).then(r=>r.json()).then(js=>{
    alert("Buddy => "+ JSON.stringify(js));
  });
}

window.onload=()=>{
  initCharts();
  refresh();
  setInterval(refresh,3000);
};
</script>
"""

NAV_MAP_HTML= r"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8"/>
<title>Navigation Map (Placeholder)</title>
<script src="https://cdn.socket.io/4.5.4/socket.io.min.js"></script>
<script src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
</head>
<body>
<h1>Navigation Map (Placeholder)</h1>
<p>In a future update, track domain transitions here in real-time.</p>
</body></html>
"""

@app.route("/")
def route_index():
    return redirect(url_for("route_dashboard"))

@app.route("/dashboard")
def route_dashboard():
    return DASHBOARD_HTML

@app.route("/nav_map")
def route_nav_map():
    return NAV_MAP_HTML

@app.route("/logs_data")
def route_logs_data():
    out= {
      "agents":{},
      "mcts_history":{},
      "emo_history":{}
    }
    for aid in agent_logs:
        out["agents"][aid] = {
          "current_url": agent_logs[aid]["current_url"],
          "cognitive_state": agent_logs[aid]["cognitive_state"],
          "cumulative_reward": agent_logs[aid]["cumulative_reward"],
          "buddy_msgs": agent_logs[aid]["buddy_msgs"]
        }
    out["mcts_history"]= {a: mcts_history[a] for a in mcts_history}
    out["emo_history"] = {a: emo_history[a]  for a in emo_history}
    return jsonify(out)

@app.route("/add-task", methods=["POST"])
def route_add_task():
    data= request.json or {}
    if "type" not in data or "content" not in data:
        return jsonify({"error":"invalid"}),400
    task_queue.append(data)
    logger.info(f"AddTask => {data}")
    return jsonify({"status":"ok","queue_len": len(task_queue)}),200

@app.route("/assign-task", methods=["POST"])
def route_assign_task():
    a_id= request.args.get("agent_id","?")
    if not task_queue:
        logger.info(f"No tasks => {a_id}")
        return jsonify({"task":None}),200
    idx=None
    for i,tk in enumerate(task_queue):
        if tk.get("agent_id") in (None,a_id):
            idx= i
            break
    if idx is not None:
        chosen= task_queue.pop(idx)
        logger.info(f"Assigned => {chosen} => {a_id}")
        return jsonify({"task":chosen}),200
    else:
        logger.info(f"No suitable tasks => {a_id}")
        return jsonify({"task":None}),200

##############################################################################
# "Conversation bounce-back" => we do a random chance to respond again
##############################################################################
def maybe_bounce_reply(to_agent, from_agent, original_msg):
    """We add a small probability that once the 'to' agent replies,
       that agent will send a new message back to 'from' => continuing the convo."""
    if random.random()< 0.4:  # 40% chance
        txt= f"Hey {from_agent}, thanks for that! Let's keep the ball rolling about: {original_msg[:100]}..."
        # We queue an agent_interaction as if 'to' is sending it
        # We'll do it in a separate thread so we don't block the main request
        def fire():
            time.sleep(0.1)  # short delay so we don't spam simultaneously
            try:
                requests.post(
                    f"http://127.0.0.1:{port_global}/agent_interaction",
                    json={"from_agent": to_agent, "to_agent": from_agent, "message": txt},
                    timeout=3
                )
            except Exception as ee:
                logger.warning(f"{to_agent} => bounce => {ee}")
        threading.Thread(target=fire,daemon=True).start()

port_global= 0  # we store the final port here

@app.route("/agent_interaction", methods=["POST"])
def route_agent_interaction():
    data= request.json or {}
    fr= data.get("from_agent","?")
    to= data.get("to_agent","?")
    msg= data.get("message","(no msg)")
    stamp= time.time()

    agent_logs[fr]["buddy_msgs"].append((stamp, f"To {to}: {msg}"))
    agent_logs[fr]["cumulative_reward"]+=2
    logger.info(f"{fr} => {to}: {msg}")

    if buddy_agents.get(to):
        # gather multi-agent context => up to last 3 lines from each
        domain= agent_logs[to]["current_url"]
        mood= agent_logs[to]["cognitive_state"]
        lines=[]
        for agid in PERSONALITIES.keys():
            recent= agent_logs[agid]["buddy_msgs"][-3:]
            for(ts, item) in recent:
                tstr= time.strftime("%H:%M:%S", time.localtime(ts))
                lines.append(f"{tstr} => [{agid}] {item}")
        combined= "\n".join(lines[-12:])
        scenario=(
            f"You are {to}, a {PERSONALITIES.get(to,'Generic')}.\n"
            f"Your current mood: {mood}\n\n"
            "Context from multiple agents (last ~12 lines):\n"
            f"{combined}\n\n"
            f"New message from {fr}: '{msg}'\n"
            "Reply in a deeper cross-talk style referencing relevant lines.\n"
            f"Optionally mention domain '{domain}'.\n"
        )
        try:
            res= Runner.run_sync(buddy_agents[to], scenario)
            reply= res.final_output
        except Exception as e:
            reply= f"(buddy fallback) => {e}"
    else:
        reply= "(buddy fallback) => no Agents or key"

    agent_logs[to]["buddy_msgs"].append((time.time(), f"From {fr}: {reply}"))
    agent_logs[to]["cumulative_reward"]+=2

    server_socketio.emit("agent_interaction",{
      "from_agent": fr,
      "to_agent": to,
      "user_msg": msg,
      "reply": reply
    }, namespace="/")

    # Possibly bounce the reply => continuing the conversation
    maybe_bounce_reply(to, fr, msg)

    return jsonify({"status":"ok","reply":reply}),200

@server_socketio.on("connect", namespace="/")
def on_connect():
    logger.info("Socket.IO client connected on / namespace.")

@server_socketio.on("disconnect", namespace="/")
def on_disconnect():
    logger.info("Socket.IO client disconnected from / namespace.")

@server_socketio.on("mcts_decision", namespace="/")
def on_mcts_decision(data):
    pass

@server_socketio.on("emo_update", namespace="/")
def on_emo_update(data):
    pass

@server_socketio.on("agent_nav_map", namespace="/")
def on_agent_nav_map(data):
    pass

##############################################################################
# The agent => partial MCTS => Poisson bursts => cross talk
##############################################################################
class AgentRNN_Burst:
    def __init__(self, agent_id, server_url):
        self.agent_id= agent_id
        self.server_url= server_url

        self.logger= logging.getLogger(agent_id)
        if not self.logger.handlers:
            sh= logging.StreamHandler(sys.stdout)
            fm= logging.Formatter("[%(asctime)s] [%(levelname)s] [%(name)s] %(message)s")
            sh.setFormatter(fm)
            self.logger.addHandler(sh)
        self.logger.setLevel(logging.INFO)

        self.sio= socketio.AsyncClient(logger=False, engineio_logger=False)

        self.burst= BurstManager()
        self.current_st= random.choice(list(CogState))
        self.last_emo= random.uniform(-0.2,0.2)

        self.link_pool= [
            "https://www.wikipedia.org/",
            "https://www.bbc.com/",
            "https://www.cnn.com/",
            "https://www.nytimes.com/",
            "https://example.com/dummy_page1"
        ]
        self.last_domain= None
        self.domain_visits= defaultdict(int)
        self.logger.info(f"{agent_id} => init done")

    async def connect_sio(self):
        @self.sio.event
        async def connect():
            self.logger.info(f"{self.agent_id} => connected => {self.server_url}")

        tries=0
        while tries<5:
            try:
                await self.sio.connect(self.server_url, socketio_path="/socket.io/", namespaces=["/"])
                return
            except Exception as e:
                self.logger.warning(f"{self.agent_id} => connect => {e}")
                tries+=1
                await asyncio.sleep(1)
        self.logger.error(f"{self.agent_id} => cannot connect after multiple attempts")

    async def run_loop(self):
        await self.connect_sio()
        while True:
            await self._step()
            await asyncio.sleep(0)  # yield

    async def _step(self):
        # 1) burst => top
        st,val= self.burst.step(context=1.0)
        if st!= self.current_st:
            self.logger.info(f"{self.agent_id} => {self.current_st.name} -> {st.name}")
        self.current_st= st
        mood= max(min(val-0.5,1),-1)
        idx= len(emo_history[self.agent_id])
        emo_history[self.agent_id].append((idx,mood))
        await self.sio.emit("emo_update",{
          "agent_id": self.agent_id,
          "x": idx,
          "emo": mood
        }, namespace="/")

        # 2) partial MCTS => random
        val2= random.uniform(0,200)
        i2= len(mcts_history[self.agent_id])
        mcts_history[self.agent_id].append((i2,val2))
        await self.sio.emit("mcts_decision",{
          "agent_id": self.agent_id,
          "x": i2,
          "cumulative_reward": val2
        }, namespace="/")

        # 3) buddy => random chance  (some calls might come from bounce)
        if random.random()< 0.4:  # 40% chance each step
            await self._buddy()

        # 4) tasks => or random navigate
        tsk= self._fetch_task()
        if tsk:
            ttype= tsk.get("type","?")
            c= tsk.get("content","")
            if ttype=="navigate":
                await self._navigate(c)
            elif ttype=="instruction":
                self.logger.info(f"{self.agent_id} => instruction => {c}")
            elif ttype=="upload":
                self.logger.info(f"{self.agent_id} => upload => {c}")
        else:
            lk= random.choice(self.link_pool)
            await self._navigate(lk)

    def _fetch_task(self):
        try:
            r= requests.post(f"{self.server_url}/assign-task?agent_id={self.agent_id}", timeout=2)
            if r.ok:
                return r.json().get("task")
        except Exception as e:
            self.logger.warning(f"{self.agent_id} => fetch_task => {e}")
        return None

    async def _buddy(self):
        # pick a different agent
        buds= list(PERSONALITIES.keys())
        if self.agent_id in buds:
            buds.remove(self.agent_id)
        if not buds: return
        to= random.choice(buds)
        mood_label= "positive" if emo_history[self.agent_id][-1][1]>=0 else "negative"
        txt= f"Hi {to}, from {self.agent_id}, I'm feeling {mood_label}, state={self.current_st.name}. Let's chat more!"
        self.logger.info(f"{self.agent_id} => buddy => {txt}")
        try:
            requests.post(
                f"{self.server_url}/agent_interaction",
                json={"from_agent":self.agent_id,"to_agent":to,"message":txt},
                timeout=2
            )
        except Exception as e:
            self.logger.warning(f"{self.agent_id} => buddy => {e}")

    async def _navigate(self, url):
        self.logger.info(f"{self.agent_id} => navigate => {url}")
        start= time.time()
        found,dom= self._fetch_links(url)

        base=0.0
        old_dom= self.last_domain
        if old_dom and dom and dom!= old_dom:
            base+= DOMAIN_HOP_REWARD
        self.last_domain= dom
        self.domain_visits[dom]+=1 if dom else 0
        base+= len(found)*0.1
        dt= time.time()-start
        rew= base* math.exp(-0.03*dt)
        agent_logs[self.agent_id]["cumulative_reward"]+= rew
        agent_logs[self.agent_id]["current_url"]= url
        agent_logs[self.agent_id]["cognitive_state"]= self.current_st.name

        if len(self.link_pool)<1000:
            for ln in found:
                if ln.startswith("http"):
                    dd= urlparse(ln).netloc
                    if self.domain_visits[dd]< MAX_DOMAIN_VISITS:
                        self.link_pool.append(ln)

        await self.sio.emit("update_agent_logs",{
          "agent_id": self.agent_id,
          "current_url": url,
          "cognitive_state": self.current_st.name,
          "cumulative_reward": agent_logs[self.agent_id]["cumulative_reward"]
        }, namespace="/")

        await self.sio.emit("agent_nav_map",{
          "agent_id": self.agent_id,
          "from_domain": old_dom if old_dom else "(none)",
          "to_domain": dom if dom else "(none)"
        }, namespace="/")

    def _fetch_links(self, url):
        try:
            r= requests.get(url, timeout=2, allow_redirects=True)
            if r.status_code!=200:
                return set(),None
            from bs4 import BeautifulSoup
            sp= BeautifulSoup(r.text,"html.parser")
            aa= sp.find_all("a", href=True)
            found= set(a["href"] for a in aa)
            d= urlparse(r.url).netloc
            return found,d
        except:
            return set(),None

    def close(self):
        try:
            asyncio.run(self.sio.disconnect())
        except:
            pass
        self.logger.info(f"{self.agent_id} closed.")

##############################################################################
# Manager
##############################################################################
class AgentManager:
    def __init__(self, agent_ids, server_url):
        self.agent_ids= agent_ids
        self.server_url= server_url
        self.agents=[]

    def start(self):
        loop= asyncio.get_event_loop()
        tasks=[]
        for aid in self.agent_ids:
            a= AgentRNN_Burst(aid, self.server_url)
            self.agents.append(a)
            tasks.append(a.run_loop())
        loop.run_until_complete(asyncio.gather(*tasks))

##############################################################################
# Startup => optional ngrok
##############################################################################
port_global= 0

def find_free_port(start=5002):
    p= start
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("127.0.0.1", p))!=0:
                return p
            p+=1

def run_server_in_background():
    global port_global
    port= find_free_port(5002)
    port_global= port
    def server_thread():
        logger.info(f"Starting server on port {port} ...")
        server_socketio.run(
            app,
            host="0.0.0.0",
            port=port,
            debug=False,
            use_reloader=False,
            allow_unsafe_werkzeug=True
        )
    th= threading.Thread(target=server_thread, daemon=True)
    th.start()
    time.sleep(2)
    return port

def start_colab_agents():
    port= run_server_in_background()
    local_url= f"http://127.0.0.1:{port}/dashboard"
    print(f"Local dashboard => {local_url}")

    token= os.getenv("NGROK_AUTH_TOKEN","")
    if token:
        try:
            from pyngrok import ngrok
            ngrok.set_auth_token(token)
            sub= os.getenv("NGROK_SUBDOMAIN","")
            if sub:
                t= ngrok.connect(addr=port, proto="http", subdomain=sub)
            else:
                t= ngrok.connect(addr=port, proto="http")
            print(f"ngrok => {t.public_url}/dashboard")
        except Exception as e:
            print("ngrok => error =>", e)
    else:
        print("No NGROK_AUTH_TOKEN => skipping ngrok.\n")

    mgr= AgentManager(["agent_1","agent_2","agent_3","agent_4"], f"http://127.0.0.1:{port}")
    mgr.start()

if __name__=="__main__":
    start_colab_agents()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://172.28.0.12:5002
INFO:werkzeug:Press CTRL+C to quit


Local dashboard => http://127.0.0.1:5002/dashboard
ngrok => https://9316de4f1aa2.ngrok.app/dashboard
[2025-05-11 02:13:20,054] [INFO] [agent_1] agent_1 => init done


INFO:agent_1:agent_1 => init done


[2025-05-11 02:13:20,056] [INFO] [agent_2] agent_2 => init done


...

INFO:agent_2:agent_2 => navigate => https://cooking.nytimes.com/recipes/1025533-creamy-cajun-style-chicken-pasta
INFO:werkzeug:127.0.0.1 - - [11/May/2025 02:16:02] "POST /agent_interaction HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2025 02:16:03] "POST /agent_interaction HTTP/1.1" 200 -


[2025-05-11 02:16:03,730] [INFO] [agent_1] agent_1 => FRUSTRATED -> NEUROTIC


INFO:agent_1:agent_1 => FRUSTRATED -> NEUROTIC


[2025-05-11 02:16:03,731] [INFO] [agent_1] agent_1 => buddy => Hi agent_4, from agent_1, I'm feeling positive, state=NEUROTIC. Let's chat more!


INFO:agent_1:agent_1 => buddy => Hi agent_4, from agent_1, I'm feeling positive, state=NEUROTIC. Let's chat more!
